In [1]:
import time

In [2]:
# pour un ordi avec GPU :

import GPUtil
import torch

In [3]:
torch.cuda.is_available()

True

In [5]:
!set CUDA_PATH=C:\Users\ehess\anaconda3\Lib\site-packages\numba\cuda

import spacy

In [8]:
# spacy.require_gpu()
# spacy.prefer_gpu()
#  j'utilise thinc.api

In [6]:
# pour un ordi avec GPU
# j'utilise thinc

# is_using_gpu = spacy.prefer_gpu()
# if is_using_gpu:
#     print("Using GPU!")
#     torch.set_default_tensor_type("torch.cuda.FloatTensor")
#     print("GPU Usage")
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  1% |  8% |


In [7]:
import pandas as pd

In [8]:
df = pd.read_csv("contrib_from_csv.csv", sep = ",", encoding = "utf-8", dtype= str)

# sample d'un petit échantillon
# df = df.sample(1000)

In [9]:
# supprimer les fin de lignes (deux types de codages)
join_breaklines = lambda x : x.replace("\x0b"," ") 
join_breaklinesn = lambda x : x.replace("\n"," ") 

# supprimer les UNK des ce corpus (trois types de codages)
kill_texte_illisible = lambda x : x.replace("texte illisible", "")
kill_illisible = lambda x : x.replace("illisible", "")
kill_illisible_plural = lambda x : x.replace("illisibles", "")
kill_illisible_capslock = lambda x : x.replace("ILLISIBLE", "")
kill_illisibleS_capslock = lambda x : x.replace("ILLISIBLES", "")

In [10]:
df["clean"] = df["Contribution"].apply(join_breaklines)
df["clean"] = df["clean"].apply(join_breaklinesn)
df["clean"] = df["clean"].apply(kill_texte_illisible)
df["clean"] = df["clean"].apply(kill_illisible)
df["clean"] = df["clean"].apply(kill_illisible_plural)
df["clean"] = df["clean"].apply(kill_illisible_capslock)
df["clean"] = df["clean"].apply(kill_illisibleS_capslock)

In [11]:
from thinc.api import set_gpu_allocator, require_gpu, prefer_gpu, use_pytorch_for_gpu_memory, set_active_gpu
set_gpu_allocator("pytorch")
import csv   

def lemmatize(doc, subset):
    '''
    Performs lemmization of input documents.
    Args:
    - docs: one column of a dataframe
    Output:
    - list (nb of contributions) of list (one cell,(size 2: with position input, lemmatized input)) of list of strings
    '''

    
    set_gpu_allocator("pytorch")
    require_gpu()
    set_active_gpu(0)
    if prefer_gpu():
        print("Using GPU!")
        torch.set_default_tensor_type("torch.cuda.FloatTensor")
        use_pytorch_for_gpu_memory()

        print("GPU Usage")
        GPUtil.showUtilization()

    nlp = spacy.load("fr_core_news_md", disable = ["parser", "ner"])

    spacy.require_gpu()

    nlp.max_length = 3000000 #this is possible as we're not using parser or ner.

    t0 = time.time()
    
    len_data = doc.shape[0]
    nb_batch = len_data // subset
    size_last_batch = len_data%subset
    first_idx = 0
    last_idx = 0
        
    for batch_nb in range(nb_batch) :
        t1 = time.time()
        lemma_text_list = []
        lemma_pos_list = []
        
        if batch_nb == nb_batch -1 :
            last_idx += size_last_batch
        else :
            first_idx = last_idx
            last_idx += subset
        
        batch = doc[first_idx : last_idx]
        
        for line in nlp.pipe(batch):
#             ta = time.time()
            lemma_pos_list.append([x.pos_ for x in line])
#             GPUtil.showUtilization()

            lemma_text_list.append([x.lemma_ for x in line])
#             tc = time.time()
#             print(f"spacy computing time per line: {tc-ta}")

        print("GPU Usage")
        GPUtil.showUtilization()
        
        with open(f'output_lemmas/lemma_temp_{batch_nb}.csv', 'w+', encoding = "utf-8") as f :
            writer = csv.writer(f, delimiter=',', quoting=csv.QUOTE_MINIMAL, escapechar = '\\')
            writer.writerow(lemma_text_list)
            f.close()
            
        with open(f'output_pos/pos_temp_{batch_nb}.csv', 'w+', encoding = "utf-8") as f :
            writer = csv.writer(f, delimiter=',', quoting=csv.QUOTE_MINIMAL, escapechar = '\\')
            writer.writerow(lemma_pos_list)
            f.close()

        
        del lemma_pos_list
        del lemma_text_list
#     for doc in nlp.pipe(doc):
#         buffer = []
#         lemma_text_list.append([[" ".join(x.pos_ for x in doc)], [" ".join(x.lemma_ for x in doc)]])     
#        on ne join() pas à ce moment là car la list reste nécessaire pour enlever les stopwords.

        t2 = time.time()
        print(f"end of batch: {batch_nb}, time: {t2-t1}")
        
    t3 = time.time()
    print("Total time: {}".format(t3-t0))

In [12]:
# GPU Total time: 130.2461621761322 (without if and with fr_core_news_md)
lemmatize(df["clean"], 64)

Using GPU!
GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 17% | 16% |
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  1% | 28% |
end of batch: 0, time: 3.2715415954589844
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 31% |
end of batch: 1, time: 2.3455281257629395
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 31% |
end of batch: 2, time: 1.888902187347412
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 35% |
end of batch: 3, time: 2.344693422317505
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  1% | 34% |
end of batch: 4, time: 1.5310380458831787
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 34% |
end of batch: 5, time: 1.764793872833252
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 34% |
end of batch: 6, time: 1.4218776226043701
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 34% |
end of batch: 7, time: 1.135420799255371
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |

GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 75, time: 1.049159049987793
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 76, time: 0.8010280132293701
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 77, time: 1.3632128238677979
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 78, time: 0.9271364212036133
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 79, time: 1.4177956581115723
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 80, time: 0.7730481624603271
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 81, time: 0.925976037979126
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 82, time: 0.9018244743347168
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 83, time: 1.2948050498962402
GPU Usage
| 

GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 150, time: 0.6509530544281006
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 151, time: 0.89925217628479
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 152, time: 1.0562026500701904
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 153, time: 0.5653417110443115
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 154, time: 0.911508321762085
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 155, time: 1.0426533222198486
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 156, time: 1.2981817722320557
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 157, time: 0.7051572799682617
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 158, time: 0.9363422393798828
GPU 

GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 224, time: 0.93536376953125
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 225, time: 0.6518030166625977
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 226, time: 0.7604212760925293
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 227, time: 0.8122744560241699
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 228, time: 1.2379565238952637
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 229, time: 0.8884108066558838
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 230, time: 0.869999885559082
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 231, time: 0.8966882228851318
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
end of batch: 232, time: 0.8372952938079834
GPU 

GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 49% |
end of batch: 298, time: 0.9330315589904785
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 49% |
end of batch: 299, time: 1.1987838745117188
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 49% |
end of batch: 300, time: 1.0129830837249756
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 49% |
end of batch: 301, time: 0.8134939670562744
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 49% |
end of batch: 302, time: 0.6593255996704102
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 49% |
end of batch: 303, time: 0.3990640640258789
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 49% |
end of batch: 304, time: 0.9030609130859375
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 49% |
end of batch: 305, time: 0.6340341567993164
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 49% |
end of batch: 306, time: 0.6030745506286621
G

...

GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3183, time: 0.7499575614929199
GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 71% | 88% |
end of batch: 3184, time: 0.34369778633117676
GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 45% | 88% |
end of batch: 3185, time: 0.2656071186065674
GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 20% | 88% |
end of batch: 3186, time: 0.20311212539672852
GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 43% | 88% |
end of batch: 3187, time: 0.2656083106994629
GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 21% | 88% |
end of batch: 3188, time: 0.2031404972076416
GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 23% | 88% |
end of batch: 3189, time: 0.2655758857727051
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3190, time: 0.5468602180480957
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  3% | 88% |
end of batch: 3191, time: 0.5468175

GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  2% | 88% |
end of batch: 3257, time: 0.6718263626098633
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3258, time: 0.7187235355377197
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3259, time: 0.5155689716339111
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3260, time: 1.124955177307129
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3261, time: 1.374878168106079
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3262, time: 1.101935625076294
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3263, time: 0.7343277931213379
GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 17% | 88% |
end of batch: 3264, time: 0.3905978202819824
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3265, time: 0.609363555908

GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 34% | 88% |
end of batch: 3331, time: 0.453096866607666
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3332, time: 0.9218132495880127
GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 10% | 88% |
end of batch: 3333, time: 0.671858549118042
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3334, time: 0.9374287128448486
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3335, time: 0.39060211181640625
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  1% | 88% |
end of batch: 3336, time: 0.5468356609344482
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3337, time: 0.48431921005249023
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3338, time: 0.4999656677246094
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3339, time: 0.812477111

GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 66% | 88% |
end of batch: 3405, time: 0.4145982265472412
GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 60% | 88% |
end of batch: 3406, time: 0.4999668598175049
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3407, time: 0.6561756134033203
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3408, time: 0.7655963897705078
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3409, time: 2.3748178482055664
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3410, time: 0.7137537002563477
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3411, time: 0.5937108993530273
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3412, time: 1.0624289512634277
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  9% | 88% |
end of batch: 3413, time: 0.421846866

GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3479, time: 0.5780856609344482
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3480, time: 1.4217720031738281
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  2% | 88% |
end of batch: 3481, time: 0.6405892372131348
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3482, time: 0.9843077659606934
GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 38% | 88% |
end of batch: 3483, time: 0.5780870914459229
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3484, time: 0.5624344348907471
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3485, time: 0.9062180519104004
GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 49% | 88% |
end of batch: 3486, time: 0.281203031539917
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 88% |
end of batch: 3487, time: 1.9026057720

In [ ]:
# précédemment, lorsque je pensais pouvoir tout stocker dans une liste :

# df["lemmatized_text"] = lemma_text_list
# df["lemmatized_pos"] = lemma_pos_list
# df.to_csv("lemmatized_df.csv", index = False)

In [ ]:
# pour afficher l'ensemble des cells, à n'utiliser que pour afficher une portion seulement

# pd.set_option('display.max_colwidth', None)